In [1]:
import pandas as pd
import numpy as np
import torch
from pickle import dump, load
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from sklearn.utils import random
from sklearn.model_selection import train_test_split
import scipy.optimize
from scipy.optimize import Bounds

import warnings
warnings.filterwarnings('ignore')

Init NN

In [2]:
class FFNetPytorch(nn.Module):
  def __init__(self, input_size, output_size, hl1=256, hl2=256):
    super(FFNetPytorch, self).__init__()
    '''
    Define the layers of the neural network. One hidden layer and output layer.
    The activation function used in between the two layers is sigmoid.
    '''
    self.layer1 = nn.Linear(input_size, hl1, bias = True)
    self.layer2 = nn.Linear(hl1, hl2, bias = True)
    self.layer3 = nn.Linear(hl2, output_size, bias=True)
    self.double()
    

  def forward(self, x):
    '''
    :param x: input to the model (N, NUM_FEATURES)

    :return:
      output: logits of the last layer of the model 
    '''
    x = torch.sigmoid(self.layer1(x))
    x = torch.sigmoid(self.layer2(x))
    x = self.layer3(x)

    return x

class CustomDataset(Dataset):
  def __init__(self, X, Y):
    self.X = X
    self.Y = Y

  def __getitem__(self, i):
    return self.X[i, ...], self.Y[i]
  
  def __len__(self):
    return self.X.shape[0]

Import Data

In [3]:
#BECAUSE NO PXPY
import pandas as pd
INPUT_COLUMNS = ["start_speed", "spin_rate", "spin_dir", "zone", "pitch_type"]
CONTEXT_COLUMNS = ["b_score", "p_score", "b_count", "s_count", "outs", "pitch_num", \
    "on_1b", "on_2b", "on_3b", "inning", "p_throws", "stand", "top"]
OUTPUT_COLUMNS = ["at_bat_score"]

combined_data = pd.read_hdf("../source_files/model_data.hdf5")
inputs = combined_data.iloc[:, 0:-1].copy()
outputs = combined_data.iloc[:, -1].copy()

inputs_t = torch.tensor(combined_data.iloc[:, 0:-1].values)
outputs_t = torch.tensor(combined_data.iloc[:, -1].values)

model = torch.load("models/HL2_100_25_TL1e3_scaled.pt")

Constraints for each type of pitch. Takes 25% and 75% Percentile of speed,
spin, and spin dir

In [4]:
convert_name = {"CH": "Changeup",
                "CU": "Curveball",
                "FC": "Cutter",
                "FF": "Four-Seam Fastball",
                "FS": "Splitter",
                "FT": "Two-Seam Fastball",
                "KC": "Knuckle Curve",
                "KN": "Knuckleball",
                "SC": "Screwball",
                "SI": "Sinker",
                "SL": "Slider"}

#Initialize constraints for pitches
pitch_constraints = {}

PITCH_TYPES = ['CH', 'CU', 'FC', 'FF', 'FS', 'FT', 'KC', 'KN', 'SI', 'SL']
ZONES = [1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14]

for pitch_type in PITCH_TYPES:
    constraints = {}
    pitches = inputs[inputs[pitch_type] == 1]

    speed_bounds = pitches["start_speed"].quantile([0.25, 0.75])
    constraints["min_speed"], constraints["max_speed"] = speed_bounds.iloc[0], speed_bounds.iloc[1]
    
    srate_bounds = pitches["spin_rate"].quantile([0.25, 0.75])
    constraints["min_srate"], constraints["max_srate"] = srate_bounds.iloc[0], srate_bounds.iloc[1]

    sdir_bounds = pitches["spin_dir"].quantile([0.25, 0.75])
    constraints["min_sdir"], constraints["max_sdir"] = sdir_bounds.iloc[0], sdir_bounds.iloc[1]

    pitch_constraints[pitch_type] = constraints


Objective function for minimizer

In [21]:
def objective(x, *args):
    model, context, pitch_type, zone, scaler = args[0], args[1], args[2], args[3], args[4]
    
    pitches = [pitch_type==val for val in PITCH_TYPES]
    pitches = pd.Series(pitches, index=PITCH_TYPES, dtype=float)

    inputs = pd.Series([x[0], x[1], x[2], zone], index=["start_speed", "spin_rate", "spin_dir", "zone"], dtype=float)

    pitch = pd.concat([inputs, pitches, context])

    pitch = scaler.transform(np.array(pitch.values).reshape(1, -1))
    #print(pitch)
    return -model(torch.tensor(pitch)).item()

Configure context and determine best pitch for given situation

In [23]:
#Initialize Context
context = {
    "b_score": 0,
    "p_score": 0,
    "b_count": 0,
    "s_count": 2,
    "outs": 0,
    "pitch_num": 3,
    "on_1b": 0,
    "on_2b": 0,
    "on_3b": 0,
    "inning": 1,
    "p_isrighty": 0,
    "b_isrighty": 0,
    "is_top_inning": 1
}
indexes = [key for key, val in context.items()]
values = [val for key, val in context.items()]
context_df = pd.Series(values, index=indexes, dtype=float)

max_score_overall = np.inf
max_pitch_overall = None

pitch_scores = []
max_score_pitch = np.inf
max_pitch_cur = None

scaler = load(open('scaler.pkl', 'rb'))

#Find optimum pitch
for pitch_type in PITCH_TYPES:
    max_score_pitch = np.inf
    max_pitch_cur = None
    bounds = Bounds([pitch_constraints[pitch_type]["min_speed"], pitch_constraints[pitch_type]["min_srate"], pitch_constraints[pitch_type]["min_sdir"]], \
            [pitch_constraints[pitch_type]["max_speed"], pitch_constraints[pitch_type]["max_srate"], pitch_constraints[pitch_type]["max_sdir"]])
    
    for zone in ZONES:
        #print(pitch_type, zone)
        args = (model, context_df, pitch_type, zone, scaler)

        x0 = [pitch_constraints[pitch_type]["max_speed"], pitch_constraints[pitch_type]["max_srate"], pitch_constraints[pitch_type]["max_sdir"]]

        optim = scipy.optimize.minimize(objective, x0, args=args,\
             method='trust-constr', bounds=bounds, tol = 1e-4, options={"maxiter": 25})

        if optim.fun < max_score_overall:
            max_score_overall = optim.fun
            max_pitch_overall = {"Pitch Type": pitch_type, \
                        "Zone": zone,
                        "Speed": optim.x[0], \
                        "Spin Rate": optim.x[1],
                        "Spin Dir": optim.x[2],
                        "Score": -optim.fun}

        if optim.fun < max_score_pitch:
            max_score_pitch = optim.fun
            max_pitch_cur = {"Pitch Type": pitch_type, \
                        "Zone": zone,
                        "Speed": optim.x[0], \
                        "Spin Rate": optim.x[1],
                        "Spin Dir": optim.x[2],
                        "Score": -optim.fun}
    
    pitch_scores.append(max_pitch_cur)

        
pitch_scores = sorted(pitch_scores, key = lambda d: d['Score'], reverse = True)

for val in pitch_scores:
    print(f"Pitch Type: {convert_name[val['Pitch Type']].ljust(18)}, Zone: {str(val['Zone']).ljust(2)}, Speed: {val['Speed']:.2f}, Spin Rate: {val['Spin Rate']:.2f}, Spin Dir: {val['Spin Dir']:.2f}, Score: {val['Score']:.3f}")

[8.6200000e+01 2.0294960e+03 2.4645475e+02 1.0000000e+00 1.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 2.0000000e+00 0.0000000e+00 3.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 1.0000000e+00 0.0000000e+00
 0.0000000e+00 1.0000000e+00]
[8.62000013e+01 2.02949600e+03 2.46454750e+02 1.00000000e+00
 1.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 2.00000000e+00 0.00000000e+00 3.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 1.00000000e+00
 0.00000000e+00 0.00000000e+00 1.00000000e+00]
[8.62000000e+01 2.02949603e+03 2.46454750e+02 1.00000000e+00
 1.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.0

KeyboardInterrupt: 